In [94]:
import numpy as np
import pandas as pd
import talib
import random
import psycopg2

In [95]:
user='pat'
password='1687'
host='34.229.138.224'
conn = psycopg2.connect(database="postgres", user=user, password=password, host=host, port="5432")
cur = conn.cursor()
#cur.execute(f"select * from master_ticker_list limit 5;")
cur.execute(f"select * from historical_stock_data where ticker_id in (select id from master_ticker_list where ticker='AAPL');")
data = cur.fetchall()
print(data[0])
conn.commit()
conn.close()


(648495289, Decimal('38.72249984741211'), Decimal('39.47999954223633'), Decimal('39.712501525878906'), Decimal('38.557498931884766'), Decimal('148158800.0'), '2019-01-02 00:00:00')


In [96]:
open = []
close = []
high = []
low = []
volume = []
date = []
columns = ['Open','High','Low','Close']
for dat in data:
    open.append(dat[1])
    close.append(dat[2])
    high.append(dat[3])
    low.append(dat[4])
    volume.append(dat[5])
    date.append(dat[6])
data = {
        'Open':open,
        'Close':close,
        'High':high,
        'Low':low,
        'Volume':volume,
        'Date':date
    }
dataset = pd.DataFrame (data, columns = columns, dtype=np.float64)
dataset.dropna()
cols = ['Open','High','Low','Close']
for b in cols:
    dataset[b].map(lambda x : float(x))
dataset

,Open,High,Low,Close
0,38.722500,39.712502,38.557499,39.480000
1,35.994999,36.430000,35.500000,35.547501
2,36.132500,37.137501,35.950001,37.064999
3,37.174999,37.207500,36.474998,36.982498
4,37.389999,37.955002,37.130001,37.687500
...,...,...,...,...
587,132.039993,134.070007,131.830002,132.539993
588,131.190002,131.490005,126.699997,127.849998
589,129.199997,130.449997,127.970001,128.100006
590,127.889999,129.750000,127.129997,129.740005


In [97]:
dataset['H-L'] = dataset['High'] - dataset['Low']
dataset['O-C'] = dataset['Close'] - dataset['Open']
dataset['3day MA'] = dataset['Close'].shift(1).rolling(window = 3).mean()
dataset['10day MA'] = dataset['Close'].shift(1).rolling(window = 10).mean()
dataset['30day MA'] = dataset['Close'].shift(1).rolling(window = 30).mean()
dataset['Std_dev']= dataset['Close'].rolling(5).std()
dataset['RSI'] = talib.RSI(dataset['Close'].values, timeperiod = 9)
dataset['Williams %R'] = talib.WILLR(dataset['High'].values, dataset['Low'].values, dataset['Close'].values, 7)
dataset

,Open,High,Low,Close,H-L,O-C,3day MA,10day MA,30day MA,Std_dev,RSI,Williams %R
0,38.722500,39.712502,38.557499,39.480000,1.155003,0.757500,NaN,NaN,NaN,NaN,NaN,NaN
1,35.994999,36.430000,35.500000,35.547501,0.930000,-0.447498,NaN,NaN,NaN,NaN,NaN,NaN
2,36.132500,37.137501,35.950001,37.064999,1.187500,0.932499,NaN,NaN,NaN,NaN,NaN,NaN
3,37.174999,37.207500,36.474998,36.982498,0.732502,-0.192501,37.364166,NaN,NaN,NaN,NaN,NaN
4,37.389999,37.955002,37.130001,37.687500,0.825001,0.297501,36.531666,NaN,NaN,1.424435,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
587,132.039993,134.070007,131.830002,132.539993,2.240005,0.500000,132.840001,133.534001,128.644000,1.122673,52.333421,-75.500234
588,131.190002,131.490005,126.699997,127.849998,4.790009,-3.340004,132.493332,133.304001,129.062333,2.358753,35.734379,-88.910315
589,129.199997,130.449997,127.970001,128.100006,2.479996,-1.099991,130.616666,132.778001,129.211000,2.577453,36.933969,-86.499443
590,127.889999,129.750000,127.129997,129.740005,2.620003,1.850006,129.496666,132.238001,129.396334,2.053293,44.569602,-70.684614


In [98]:
#training for network

#tells close price tomorrow greater than today
dataset['Price_Rise'] = np.where(dataset['Close'].shift(-1) > dataset['Close'], 1, 0)

dataset = dataset.dropna()

#input and output variable storage
X = dataset.iloc[:, 4:-1]
y = dataset.iloc[:, -1]

split = int(len(dataset)*0.8)
X_train, X_test, y_train, y_test = X[:split], X[split:], y[:split], y[split:]

In [99]:
#dataset preprocessing and normalizing
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [100]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

classifier = Sequential()

#hidden layers
classifier.add(Dense(units = 128, kernel_initializer = 'uniform', activation = 'relu', input_dim = X.shape[1]))

classifier.add(Dense(units = 128, kernel_initializer = 'uniform', activation = 'relu'))

#output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

classifier.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['accuracy'])

classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
45/45 [==============================] - 1s 2ms/step - loss: 0.2502 - accuracy: 0.4961
Epoch 2/100
45/45 [==============================] - 0s 2ms/step - loss: 0.2472 - accuracy: 0.5737
Epoch 3/100
45/45 [==============================] - 0s 2ms/step - loss: 0.2450 - accuracy: 0.5848
Epoch 4/100
45/45 [==============================] - 0s 2ms/step - loss: 0.2417 - accuracy: 0.5757
Epoch 5/100
45/45 [==============================] - 0s 2ms/step - loss: 0.2453 - accuracy: 0.5701
Epoch 6/100
45/45 [==============================] - 0s 2ms/step - loss: 0.2382 - accuracy: 0.6173
Epoch 7/100
45/45 [==============================] - 0s 2ms/step - loss: 0.2443 - accuracy: 0.5614
Epoch 8/100
45/45 [==============================] - 0s 2ms/step - loss: 0.2344 - accuracy: 0.6335
Epoch 9/100
45/45 [==============================] - 0s 2ms/step - loss: 0.2457 - accuracy: 0.5767
Epoch 10/100
45/45 [==============================] - 0s 2ms/step - loss: 0.2327 - accuracy: 0.6151
Epoch 11/

45/45 [==============================] - 0s 2ms/step - loss: 0.1463 - accuracy: 0.7946
Epoch 84/100
45/45 [==============================] - 0s 3ms/step - loss: 0.1410 - accuracy: 0.8022
Epoch 85/100
45/45 [==============================] - 0s 2ms/step - loss: 0.1368 - accuracy: 0.8187
Epoch 86/100
45/45 [==============================] - 0s 2ms/step - loss: 0.1426 - accuracy: 0.7940
Epoch 87/100
45/45 [==============================] - 0s 2ms/step - loss: 0.1446 - accuracy: 0.8105
Epoch 88/100
45/45 [==============================] - 0s 2ms/step - loss: 0.1442 - accuracy: 0.8132
Epoch 89/100
45/45 [==============================] - 0s 2ms/step - loss: 0.1481 - accuracy: 0.8012
Epoch 90/100
45/45 [==============================] - 0s 2ms/step - loss: 0.1493 - accuracy: 0.7753
Epoch 91/100
45/45 [==============================] - 0s 2ms/step - loss: 0.1491 - accuracy: 0.8015
Epoch 92/100
45/45 [==============================] - 0s 2ms/step - loss: 0.1405 - accuracy: 0.7915
Epoch 93/100


In [101]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

dataset['y_pred'] = np.NaN
dataset.iloc[(len(dataset) - len(y_pred)):,-1:] = y_pred
trade_dataset = dataset.dropna()

trade_dataset

<ipython-input-101-c2a7ca5f1f91>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['y_pred'] = np.NaN
/home/pat/Desktop/algo_neural_net/algo_strat_neural_network/venv/lib/python3.8/site-packages/pandas/core/indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)


,Open,High,Low,Close,H-L,O-C,3day MA,10day MA,30day MA,Std_dev,RSI,Williams %R,Price_Rise,y_pred
479,113.910004,115.849998,112.589996,115.169998,3.260002,1.259995,116.609998,118.148000,116.662667,2.022457,43.909134,-69.285698,1,False
480,115.550003,116.750000,115.169998,116.029999,1.580002,0.479996,115.453331,118.067999,116.465000,1.861808,47.239359,-57.425722,1,False
481,116.570000,117.489998,116.220001,116.589996,1.269997,0.019997,115.016665,117.721999,116.293000,1.346097,49.438443,-44.674991,1,False
482,116.970001,120.970001,116.809998,119.050003,4.160004,2.080002,115.929998,117.459999,116.155666,1.927933,58.074453,-22.911659,1,True
483,121.010002,123.470001,120.010002,122.720001,3.459999,1.709999,117.223333,117.438999,116.156667,3.050415,67.415337,-6.893379,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
587,132.039993,134.070007,131.830002,132.539993,2.240005,0.500000,132.840001,133.534001,128.644000,1.122673,52.333421,-75.500234,0,False
588,131.190002,131.490005,126.699997,127.849998,4.790009,-3.340004,132.493332,133.304001,129.062333,2.358753,35.734379,-88.910315,1,False
589,129.199997,130.449997,127.970001,128.100006,2.479996,-1.099991,130.616666,132.778001,129.211000,2.577453,36.933969,-86.499443,1,True
590,127.889999,129.750000,127.129997,129.740005,2.620003,1.850006,129.496666,132.238001,129.396334,2.053293,44.569602,-70.684614,1,False


In [102]:
trade_dataset['Tomorrows Returns'] = 0.
trade_dataset['Tomorrows Returns'] = np.log(trade_dataset['Close']/trade_dataset['Close'].shift(1))
trade_dataset['Tomorrows Returns'] = trade_dataset['Tomorrows Returns'].shift(-1)

trade_dataset

<ipython-input-102-297bff81a4ab>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trade_dataset['Tomorrows Returns'] = 0.
<ipython-input-102-297bff81a4ab>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trade_dataset['Tomorrows Returns'] = np.log(trade_dataset['Close']/trade_dataset['Close'].shift(1))
<ipython-input-102-297bff81a4ab>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

,Open,High,Low,Close,H-L,O-C,3day MA,10day MA,30day MA,Std_dev,RSI,Williams %R,Price_Rise,y_pred,Tomorrows Returns
479,113.910004,115.849998,112.589996,115.169998,3.260002,1.259995,116.609998,118.148000,116.662667,2.022457,43.909134,-69.285698,1,False,0.007439
480,115.550003,116.750000,115.169998,116.029999,1.580002,0.479996,115.453331,118.067999,116.465000,1.861808,47.239359,-57.425722,1,False,0.004815
481,116.570000,117.489998,116.220001,116.589996,1.269997,0.019997,115.016665,117.721999,116.293000,1.346097,49.438443,-44.674991,1,False,0.020880
482,116.970001,120.970001,116.809998,119.050003,4.160004,2.080002,115.929998,117.459999,116.155666,1.927933,58.074453,-22.911659,1,True,0.030362
483,121.010002,123.470001,120.010002,122.720001,3.459999,1.709999,117.223333,117.438999,116.156667,3.050415,67.415337,-6.893379,1,False,0.002929
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
587,132.039993,134.070007,131.830002,132.539993,2.240005,0.500000,132.840001,133.534001,128.644000,1.122673,52.333421,-75.500234,0,False,-0.036027
588,131.190002,131.490005,126.699997,127.849998,4.790009,-3.340004,132.493332,133.304001,129.062333,2.358753,35.734379,-88.910315,1,False,0.001954
589,129.199997,130.449997,127.970001,128.100006,2.479996,-1.099991,130.616666,132.778001,129.211000,2.577453,36.933969,-86.499443,1,True,0.012721
590,127.889999,129.750000,127.129997,129.740005,2.620003,1.850006,129.496666,132.238001,129.396334,2.053293,44.569602,-70.684614,1,False,0.003616
